In [31]:
import re
import lyricsgenius
import pandas as pd
from textblob import TextBlob, Word
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import wikipedia
from collections import Counter

# 1

In [2]:
genius = lyricsgenius.Genius("PAHsckOS0w26KxYxe3R_1U7y87eXsYhiTe9ifRl89WbmImP_suZsw3MVLqDRGYaw")
genius.excluded_terms = ["(Remix)", "(Live)"] 

In [3]:
my_artists = ["Beyonce","Bruno Mars","John Legend"]

In [5]:
songs = []
for i in my_artists:
    artist = genius.search_artist(i, max_songs=5, sort="popularity")
    songs.append(artist.songs)

Searching for songs by Beyonce...

Changing artist name to 'Beyoncé'
Song 1: "Drunk in Love"
Song 2: "FORMATION"
Song 3: "Partition"
Song 4: "Mine"
Song 5: "HOLD UP"

Reached user-specified song limit (5).
Done. Found 5 songs.
Searching for songs by Bruno Mars...

Song 1: "That’s What I Like"
"Finesse (Remix)" is not valid. Skipping.
Song 2: "Versace on the Floor"
Song 3: "24K Magic"
Song 4: "When I Was Your Man"
Song 5: "Finesse"

Reached user-specified song limit (5).
Done. Found 5 songs.
Searching for songs by John Legend...

Song 1: "All of Me"
Song 2: "You & I (Nobody in the World)"
"POWER (Remix)" is not valid. Skipping.
Song 3: "Love Me Now"
"All of the Lights (Remix)" is not valid. Skipping.
Song 4: "Made to Love"
Song 5: "Ordinary People"

Reached user-specified song limit (5).
Done. Found 5 songs.


In [6]:
artists = []
for n in range(3):
    artists += [songs[n][i].artist for i in range(5)]

In [7]:
artists

['Beyoncé',
 'Beyoncé',
 'Beyoncé',
 'Beyoncé',
 'Beyoncé',
 'Bruno Mars',
 'Bruno Mars',
 'Bruno Mars',
 'Bruno Mars',
 'Bruno Mars',
 'John Legend',
 'John Legend',
 'John Legend',
 'John Legend',
 'John Legend']

In [8]:
titles = []
for n in range(3):
    titles += [songs[n][i].title for i in range(5)]

In [9]:
titles

['Drunk in Love',
 'FORMATION',
 'Partition',
 'Mine',
 'HOLD UP',
 'That’s What I Like',
 'Versace on the Floor',
 '24K Magic',
 'When I Was Your Man',
 'Finesse',
 'All of Me',
 'You & I (Nobody in the World)',
 'Love Me Now',
 'Made to Love',
 'Ordinary People']

# 1-a 

In [10]:
lyrics = []
for n in range(3):
    lyrics += [songs[n][i].lyrics for i in range(5)]

In [54]:
#print(lyrics)

In [12]:
years = []
for n in range(3):
    years += [songs[n][i].year for i in range(5)]

In [13]:
only_years = [years[i][:4] for i in range(len(years))]

In [14]:
only_years

['2013',
 '2016',
 '2013',
 '2013',
 '2016',
 '2016',
 '2016',
 '2016',
 '2013',
 '2016',
 '2013',
 '2013',
 '2016',
 '2013',
 '2004']

In [15]:
def get_score(text):
    sentiment = TextBlob(text).sentiment[0]
    return sentiment

In [16]:
sentiment_score = [get_score(lyrics[i]) for i in range(len(lyrics))]

In [17]:
sentiment_score

[0.17728696741854635,
 0.3023301801079579,
 0.14489417989417988,
 0.1773511904761903,
 0.14553298383573612,
 0.3537878787878787,
 0.14249084249084248,
 0.12333762333762333,
 0.03156565656565657,
 0.2625850340136055,
 0.2622420634920635,
 0.24285714285714285,
 0.40787190845330384,
 0.4005952380952381,
 -0.14222190852625632]

# 1-b

In [18]:
df = pd.DataFrame(columns=['Artist', 'Song', 'Lyrics','Year','Sentiment_Score'])

In [19]:
df.Artist = artists

In [20]:
df.Song = titles

In [21]:
df.Lyrics = lyrics

In [22]:
df.Year = only_years

In [23]:
df.Sentiment_Score = sentiment_score

In [24]:
df

,Artist,Song,Lyrics,Year,Sentiment_Score
0,Beyoncé,Drunk in Love,"[Verse 1: Beyoncé]\nI've been drinkin', I've b...",2013,0.177287
1,Beyoncé,FORMATION,[Intro: Messy Mya]\nWhat happened at the New W...,2016,0.302330
2,Beyoncé,Partition,[Part 1: Yoncé]\n\n[Intro]\nLet me hear you sa...,2013,0.144894
3,Beyoncé,Mine,"[Produced by Noah “40” Shebib, Beyoncé Knowles...",2013,0.177351
4,Beyoncé,HOLD UP,"[Chorus]\nHold up, they don't love you like I ...",2016,0.145533
5,Bruno Mars,That’s What I Like,"[Verse 1]\nHey, hey, hey\nI got a condo in Man...",2016,0.353788
6,Bruno Mars,Versace on the Floor,"[Verse 1]\nLet's take our time tonight, girl\n...",2016,0.142491
7,Bruno Mars,24K Magic,[Intro]\nTonight\nI just want to take you high...,2016,0.123338
8,Bruno Mars,When I Was Your Man,[Verse 1]\nSame bed but it feels just a little...,2013,0.031566
9,Bruno Mars,Finesse,"[Verse 1]\nOoh, don't we look good together?\n...",2016,0.262585


# 1-c Most positive 2 songs : 1- John Legend, Love Me Now, 2- John Legend, Made to Love
# Most negative 2 songs : 1- John Legend, Ordinary People, 2- Bruno Mars, When I Was Your Man

In [25]:
most_positive = df.sort_values(by = "Sentiment_Score",ascending=False).head(2)

In [26]:
most_positive

,Artist,Song,Lyrics,Year,Sentiment_Score
12,John Legend,Love Me Now,"[Verse 1]\nPulling me further, further than I'...",2016,0.407872
13,John Legend,Made to Love,"[Produced by Dave Tozer, Da Internz, Kanye Wes...",2013,0.400595


In [27]:
most_negative = df.sort_values(by = "Sentiment_Score",ascending=True).head(2)

In [28]:
most_negative

,Artist,Song,Lyrics,Year,Sentiment_Score
14,John Legend,Ordinary People,"[Intro]\nOh, oh, oh, oh\n\n[Verse 1]\nGirl, I'...",2004,-0.142222
8,Bruno Mars,When I Was Your Man,[Verse 1]\nSame bed but it feels just a little...,2013,0.031566


# 2

In [38]:
wikipedia.set_lang("ru")

In [39]:
summary = [wikipedia.summary(i) for i in my_artists]

# 2-a

In [40]:
summary

['Бейо́нсе Жизе́ль Но́улз-Картер (англ. Beyoncé Giselle Knowles-Carter; род. 4 сентября 1981, Хьюстон), более известная просто как Бейо́нсе (произносится /biˈjɒnseɪ/ би-йон-сей) — американская певица в стиле R’n’B, актриса, танцовщица, музыкальный продюсер. Ещё будучи ребёнком, участвовала в разных вокальных и танцевальных конкурсах, в школе также принимала участие в различных художественных выступлениях. Ноулз прославилась в конце 1990-х годов, будучи солисткой женской R&B группы Destiny’s Child.\nВо время раскола Destiny’s Child Ноулз выпустила свой дебютный сольный альбом Dangerously in Love (2003), в который входили такие хиты, как «Crazy in Love» и «Baby Boy», и который стал одним из самых успешных альбомов 2003 года. За этот альбом Ноулз получила рекордные пять «Грэмми».\nВ 2006 году, спустя год после окончательного развала группы, Бейонсе выпустила альбом B’Day, который занял первую строку в чартах Billboard и включал хиты «Déjà Vu», «Irreplaceable» и «Beautiful Liar». Её третий

In [41]:
sums_blob = [TextBlob("".join(summary[i])) for i in range(3)]

In [42]:
sums_blob = [sums_blob[i].replace("году","") for i in range(3)]

In [43]:
words = [sums_blob[i].words.lower() for i in range(3)]

In [44]:
sw = stopwords.words("russian")

In [45]:
without_sw = []
for n in range(3):
    without_sw.append([i for i in words[n] if i not in sw and i.isalnum()])

# 2-b

In [46]:
most_common = [Counter(without_sw[i]).most_common(5) for i in range(3)]

In [47]:
most_common

[[('ноулз', 9), ('альбом', 5), ('самых', 5), ('группы', 4), ('destiny', 4)],
 [('марс', 7), ('the', 5), ('billboard', 5), ('грэмми', 5), ('марса', 4)],
 [('джон', 2), ('англ', 2), ('john', 2), ('roger', 1), ('stephens', 1)]]

# 3

In [48]:
class human:
    nationality = "Armenian"
    def __init__(self,name,surname,profession,salary):
        self.name = name
        self.surname = surname
        self.profession = profession
        self.salary = salary
    def desc(self):
        return "Hi, I am " + self.name + " " + self.surname + ". I am a/an " + self.profession + " and I earn " + self.salary + " a month."

In [49]:
Aram = human("Aram","Sahakyan","actor","2000 USD")

In [50]:
Aram.nationality

'Armenian'

In [51]:
Aram.desc()

'Hi, I am Aram Sahakyan. I am a/an actor and I earn 2000 USD a month.'

In [52]:
Aram.profession

'actor'